In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import math

In [2]:
dist = pd.read_csv('dist_data.csv', sep=';')

In [3]:
dist_clean = dist[dist['dist'] != "[[], []]"]

In [4]:
x = dist_clean.iloc[:, :-1].values
y = dist_clean.iloc[:, 1].values

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30)

In [6]:
def clean_sample(sample):
    sample_list = sample[0].replace(', [', '; [').split(';')
    sample_x = sample_list[0]
    sample_Px = sample_list[1]

    sample_x_list = []
    sample_Px_list = []
    
    for x in sample_x.split(','):
        sample_x_list.append(x.replace(' ', '').replace('[', '').replace(']', ''))
    for Px in sample_Px.split(','):
        sample_Px_list.append(Px.replace(' ', '').replace('[', '').replace(']', ''))

    dict_sample = {}
    for i in range(len(sample_x_list)):
        dict_sample[sample_x_list[i]] = sample_Px_list[i]
    return dict_sample

In [7]:
def all_keys_dataset1_exist_in_dataset2(dataset1, dataset2, threshold):
    count = 0
    total = len(dataset1.keys())
    for key in dataset1.keys():
        if dataset2.get(key) is None:
            pass
        else:
            count += 1

    if (count/total) * 100 >= float(threshold):
        #print(count, (count/total) * 100, float(threshold))
        return True
    else:
        return False

In [8]:
def calculate_distance(distance_func, min_precent_similarity):
    # count = 0
    result = []
    for i, sample_x_test in enumerate(x_test): 
        result_zikma = []
    
        for j, sample_x in enumerate(x):
            if all_keys_dataset1_exist_in_dataset2(clean_sample(sample_x_test), clean_sample(sample_x), min_precent_similarity):
                Px_list = []
                for xd1, Pxd1 in clean_sample(sample_x_test).items():
                    if xd1 == "" or Pxd1 == "":
                        continue
                    Pxd2 = clean_sample(sample_x).get(xd1)
                    if Pxd2 is None:
                        return False
                    #print(xd1, Pxd1, Pxd2)
                    Px_list.append([Pxd1, Pxd2])

                
                sum = distance_func(Px_list)
                result_zikma.append([y[j], sum])
                #print(Px_list, sum)
                
            else:
                continue
        
        result.append([sample_x_test ,result_zikma])
        
        # count += 1
        # if count > 10:
        #     break
    
    return result

In [9]:
def js(Px_list):
    for i in Px_list:
        Mx = (float(i[0]) + float(i[1]))/2
        i.append(Mx)

    PM_list = []
    QM_list = []
    for i in Px_list:
        PM_list.append([i[0], i[2]])
        QM_list.append([i[1], i[2]])

    sum1 = kl(PM_list)
    sum2 = kl(QM_list)

    return (sum1 + sum2)/2

In [10]:
def kl(Px_list):
    sum = 0 
    for i in Px_list:
        Pxd1 = i[0]
        Pxd2 = i[1]
        sum += float(Pxd1) * math.log10(float(Pxd1) / float(Pxd2))
    return sum

In [12]:
def Predict(all_distance_label, threshold):
    y_pred = []
    for i in range(len(x_test)):
        t = pd.DataFrame(all_distance_label[i][1], columns=['Label', 'Kl Result'])
        tmp = t[t['Kl Result'] < threshold] #0.99 - 1.01
        grouped_df = tmp.groupby(['Label']).size().reset_index(name='Count')
        sorted_df = grouped_df.sort_values(by=['Count'], ascending=False)
        
        if tmp.values.size == 0:
            y_pred.append(0)
        else:
            y_pred.append(sorted_df.head(1)['Label'].values[0])
    return y_pred

In [13]:
def diffrence_pred_test(y_pred, y_test):
    test = []
    count = 0
    for i, value1 in enumerate(y_pred):
        if value1 != y_test[i]:
            test.append([i, value1, y_test[i]])
            count+=1
    
    return pd.DataFrame(test, columns=['index', 'y_pred', 'y_test'])

In [15]:
final_result_kl_full = calculate_distance(kl, 100)

In [61]:
y_pred_kl = Predict(final_result_kl_full, 0.99) #good threshold 0.99 - 1.01

In [62]:
diffrence_pred_test(y_pred_kl, y_test)

,index,y_pred,y_test
0,4,27525,27321
1,23,27523,27321
2,24,27523,27311
3,35,27321,27525
4,41,27321,27523
...,...,...,...
121,1536,27525,27341
122,1537,27525,27652
123,1545,27525,27523
124,1571,27321,27525


In [18]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
res0 = confusion_matrix(y_test, y_pred_kl)
print("Confusion Matrix:")
print(res0)
res01 = classification_report(y_test, y_pred_kl)
print("Classification Report:",)
print (res01)
res02 = accuracy_score(y_test,y_pred_kl)
print("Accuracy:",res02)

Confusion Matrix:
[[131   0   0   2   0   0   0]
 [  0 367   0  24   8   0   0]
 [  0   0   0   2  31   0   0]
 [  0   8   0 287  18   0   0]
 [  0  23   0   0 559   0   0]
 [  0   0   0   9   1  14   0]
 [  0   0   0   0   0   0  98]]
Classification Report:
              precision    recall  f1-score   support

       27311       1.00      0.98      0.99       133
       27321       0.92      0.92      0.92       399
       27341       0.00      0.00      0.00        33
       27523       0.89      0.92      0.90       313
       27525       0.91      0.96      0.93       582
       27652       1.00      0.58      0.74        24
       28090       1.00      1.00      1.00        98

    accuracy                           0.92      1582
   macro avg       0.82      0.77      0.78      1582
weighted avg       0.90      0.92      0.91      1582

Accuracy: 0.9203539823008849


/home/arminjp/Documents/Project/pandas/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arminjp/Documents/Project/pandas/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arminjp/Documents/Project/pandas/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,